In [1]:

import os
import numpy as np
import torch
from tqdm import tqdm
import json
from dual_network import Dual3DCNN3, Dual3DCNN4, Dual3DCNN5, FlexibleDual3DCNN
import torch
from torch.utils.data import Dataset
import SimpleITK as sitk
import numpy as np
import glob
from utilities import create_list_from_master_json, read_json_file, split_data
import re
import glob
import random
from torch.utils.data import Dataset, DataLoader
import numpy as np
import SimpleITK as sitk
import torch
from utilities import list_patient_folders, prepare_data_nrrd, split_data
from monai.transforms import Compose, LoadImaged, EnsureChannelFirstd, Spacingd, ScaleIntensityd, SpatialPadd, CenterSpatialCropd, ScaleIntensityRanged
from monai.data import CacheDataset, DataLoader, Dataset
from monai.transforms import LoadImaged
from monai.data.image_reader import ITKReader
from monai.data import SmartCacheDataset
import random
import optuna
from optuna.integration import PyTorchLightningPruningCallback

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [3]:
# data_path_NEW = '/data/shahpouriz/DBP_newDATA/nrrd/oneCTperPatients/proton'
# data_path = '/data/shahpouriz/DBP_DATA_total/nrrd/oneCTperPatinet/proton'
data_path = '/data/shahpouriz/DBP_DATA_total/nrrd/oneOPTZperPatinet/proton'

patient_list_NEW = list_patient_folders(data_path)
# Shuffle patient list if you want randomness
random.seed(42)  # You can choose any number as the seed
random.shuffle(patient_list_NEW)

# Define split sizes
total_patients = len(patient_list_NEW)
train_size = int(total_patients * 0.70)
val_size = int(total_patients * 0.20)
# The rest will be for the test set

# Split the patient list
train_patients = patient_list_NEW[:train_size]
val_patients = patient_list_NEW[train_size:train_size + val_size]
test_patients = patient_list_NEW[train_size + val_size:]

train_pct, train_rct, train_pos = prepare_data_nrrd(data_path, train_patients)
val_pct, val_rct, val_pos = prepare_data_nrrd(data_path, val_patients)
test_pct, test_rct, test_pos = prepare_data_nrrd(data_path, test_patients)

# Create dictionaries for each dataset
train_data = [{"plan": img, "repeat": tar, "pos": pos} for img, tar, pos in zip(train_pct, train_rct, train_pos)]
val_data = [{"plan": img, "repeat": tar, "pos": pos} for img, tar, pos in zip(val_pct, val_rct, val_pos)]
test_data = [{"plan": img, "repeat": tar, "pos": pos} for img, tar, pos in zip(test_pct, test_rct, test_pos)]


# Check the lengths of the sets
print("Number of training samples:", len(train_data))
print("Number of validation samples:", len(val_data))
print("Number of test samples:", len(test_data))
print(len(test_data)+len(val_data)+len(train_data))

Number of training samples: 336
Number of validation samples: 102
Number of test samples: 65
503


In [4]:

dim = 128
size = (dim, dim, dim)
pixdim = (3.0, 3.0, 3.0)
transforms = Compose([
        LoadImaged(keys=["plan", "repeat"], reader=ITKReader()),
        EnsureChannelFirstd(keys=["plan", "repeat"]),
        # ScaleIntensityd(keys=["plan", "repeat"]),
        Spacingd(keys=["plan", "repeat"], pixdim=pixdim, mode='trilinear'),
        SpatialPadd(keys=["plan", "repeat"], spatial_size=size, mode='constant'),  # Ensure minimum size
        CenterSpatialCropd(keys=["plan", "repeat"], roi_size=size),  # Ensure uniform size
    ])


train_ds = CacheDataset(data=train_data, transform=transforms, cache_rate=0.8, num_workers=1)
train_loader = DataLoader(train_ds, batch_size=1, shuffle=True, num_workers=1)

val_ds = CacheDataset(data=val_data, transform=transforms, cache_rate=0.8, num_workers=1)
val_loader = DataLoader(val_ds, batch_size=1, shuffle=False, num_workers=1)


Loading dataset: 100%|██████████| 81/81 [00:03<00:00, 21.90it/s]


In [5]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import optuna
from optuna.trial import TrialState
import functools
from torch.optim.lr_scheduler import ReduceLROnPlateau


lambda_reg = 0.0  # for L1 regularization
weight_decay = 0.0  # for L2 regularization


def objective(trial, device, final_epoch):
    # Predefined layer configurations
    architectures = [
        [8, 16, 32],   # Small network
        [16, 32, 64],  # Small network

        [32, 64, 128],  # Medium network
        [16, 32, 64, 128],

        [32, 64, 128, 256],  # Larger network
        [16, 32, 128, 256, 512]  # Even larger network
    ]
    chosen_architecture = trial.suggest_categorical('architecture', architectures)
    
    model = FlexibleDual3DCNN(chosen_architecture).to(device)
    
    # Hyperparameters to optimize
    lr = trial.suggest_categorical('lr', [1e-5, 1e-4, 1e-3, 1e-2, 1e-1])
    optimizer_name = trial.suggest_categorical('optimizer', ['Adam']) #, 'RMSprop', 'SGD'])
    loss_function = trial.suggest_categorical('loss_function', ['MSELoss', 'L1Loss']) #, 'SmoothL1Loss'])

    # Select optimizer
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=4, verbose=True)



    # Select loss function
    if loss_function == 'MSELoss':
        criterion = nn.MSELoss()
    elif loss_function == 'L1Loss':
        criterion = nn.L1Loss()
    elif loss_function == 'SmoothL1Loss':
        criterion = nn.SmoothL1Loss()



    # Training loop
    model.train()

    for epoch in range(final_epoch):
        model.train()
        loss_list = []
        for batch_data in train_loader:
            pCT, rCT = batch_data["plan"].to(device), batch_data["repeat"].to(device)
            reg = batch_data["pos"].clone().detach().requires_grad_(True).to(device)  # If gradients are required for 'reg'
            optimizer.zero_grad()

            output = model(pCT, rCT)
            loss = criterion(output, reg)


            # L1 Regularization
            l1_reg = torch.tensor(0., requires_grad=True).to(device)
            for name, param in model.named_parameters():
                # print(name, param)
                if 'weight' in name:  # Apply L1 only to weights, not biases
                    l1_reg = l1_reg + torch.norm(param, 1)
            
            # Combine loss with L1 regularization
            loss = loss + lambda_reg * l1_reg


            loss.backward()
            optimizer.step()

            loss_list.append(loss.item())
            mean_tot_loss = np.mean(loss_list)
            # print(f'Epoch: {epoch}/{final_epoch}, Batch: {i+1}/{len(train_loader)}, Loss_avg: {mean_tot_loss}')


        # Validation loop
        model.eval()
        val_loss = []
        with torch.no_grad():
            for batch_data in val_loader:
                pCT_val, rCT_val = batch_data["plan"].to(device), batch_data["repeat"].to(device)
                reg_val = batch_data["pos"].clone().detach().requires_grad_(True).to(device)  # If gradients are required for 'reg'

                output_val = model(pCT_val, rCT_val)
                loss_output_val = criterion(output_val, reg_val)

                val_loss.append(loss_output_val.item())

            mean_val_loss = np.mean(val_loss)
            # print(f'Epoch [{epoch+1}/{final_epoch}], Validation Loss: {mean_val_loss:.4f}')

            # Adjust learning rate
            scheduler.step(mean_val_loss)
    return mean_val_loss


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

final_epoch = 20 # The number of epochs



study = optuna.create_study(direction='minimize', study_name='DBP_optimize_1opt', storage='sqlite:///DBP_optimizer_1opt.db', load_if_exists=True)
study.optimize(lambda trial: objective(trial, device, final_epoch), n_trials=140) # the number of trials as needed

print("Best trial:")
print("  Loss: ", study.best_trial.value)
print("  Params: ")
for key, value in study.best_trial.params.items():
    print(f"    {key}: {value}")


[I 2024-03-18 15:28:13,501] A new study created in RDB with name: DBP_optimize_1opt
Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [8, 16, 32] which is of type list.
Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [16, 32, 64] which is of type list.
Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [32, 64, 128] which is of type list.
Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [16, 32, 64, 128] which is of type list.
Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains [32, 64, 128, 256] which is of type list.
Choices for a categorical distribution should be a tuple of None, bool, int, float and str

Epoch 00020: reducing learning rate of group 0 to 1.0000e-05.
Epoch 00009: reducing learning rate of group 0 to 1.0000e-03.
Epoch 00014: reducing learning rate of group 0 to 1.0000e-04.
Epoch 00019: reducing learning rate of group 0 to 1.0000e-05.


[I 2024-03-18 16:41:09,179] Trial 1 finished with value: 0.6658520485547271 and parameters: {'architecture': [16, 32, 64], 'lr': 0.01, 'optimizer': 'Adam', 'loss_function': 'MSELoss'}. Best is trial 1 with value: 0.6658520485547271.


Epoch 00008: reducing learning rate of group 0 to 1.0000e-04.
Epoch 00013: reducing learning rate of group 0 to 1.0000e-05.
Epoch 00018: reducing learning rate of group 0 to 1.0000e-06.


[I 2024-03-18 17:04:41,216] Trial 2 finished with value: 0.6549073285331913 and parameters: {'architecture': [16, 32, 128, 256, 512], 'lr': 0.001, 'optimizer': 'Adam', 'loss_function': 'L1Loss'}. Best is trial 2 with value: 0.6549073285331913.
[I 2024-03-18 17:24:55,490] Trial 3 finished with value: 0.6587223221054849 and parameters: {'architecture': [16, 32, 128, 256, 512], 'lr': 1e-05, 'optimizer': 'Adam', 'loss_function': 'L1Loss'}. Best is trial 2 with value: 0.6549073285331913.


...

Epoch 00020: reducing learning rate of group 0 to 1.0000e-06.
Epoch 00016: reducing learning rate of group 0 to 1.0000e-04.


[I 2024-03-19 09:21:06,457] Trial 138 finished with value: 0.6552377059763553 and parameters: {'architecture': [8, 16, 32], 'lr': 0.001, 'optimizer': 'Adam', 'loss_function': 'L1Loss'}. Best is trial 93 with value: 0.654657388306862.


Epoch 00009: reducing learning rate of group 0 to 1.0000e-04.
Epoch 00014: reducing learning rate of group 0 to 1.0000e-05.
Epoch 00019: reducing learning rate of group 0 to 1.0000e-06.


[I 2024-03-19 09:25:19,662] Trial 139 finished with value: 0.6550741263427863 and parameters: {'architecture': [8, 16, 32], 'lr': 0.001, 'optimizer': 'Adam', 'loss_function': 'L1Loss'}. Best is trial 93 with value: 0.654657388306862.


Best trial:
  Loss:  0.654657388306862
  Params: 
    architecture: [8, 16, 32]
    lr: 0.001
    optimizer: Adam
    loss_function: L1Loss
